In [1]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import tensorflow as tf
from transformers import BertTokenizer

In [2]:
df=pd.read_csv('final_animal_description.csv')
df.head()

,Disease Description,Disease
0,"Fever, Sore throat, Cough, Headache, Muscle p...",Avian Influenza
1,"fever , huffy throat , cough , headache , braw...",Avian Influenza
2,"fever , sensitive throat , cough , concern , m...",Avian Influenza
3,"fever , afflictive throat , cough , headache ,...",Avian Influenza
4,"fever , sore throat , cough , concern , muscle...",Avian Influenza


In [3]:
df.info

<bound method DataFrame.info of                                     Disease Description          Disease
0     Fever,  Sore throat, Cough, Headache, Muscle p...  Avian Influenza
1     fever , huffy throat , cough , headache , braw...  Avian Influenza
2     fever , sensitive throat , cough , concern , m...  Avian Influenza
3     fever , afflictive throat , cough , headache ,...  Avian Influenza
4     fever , sore throat , cough , concern , muscle...  Avian Influenza
...                                                 ...              ...
1195  Swollen eyelids eyelids , Eyes closed , Oral l...        Avian Pox
1196  Swollen eyelids , Eyes lesions closed , Oral l...        Avian Pox
1197  Swollen eyelids, Eyes closed, Oral lesions, We...        Avian Pox
1198  Swollen eyelids , Eyes Eyes closed , Oral lesi...        Avian Pox
1199  Swollen , Eyes closed , lesions , Weight loss ...        Avian Pox

[1200 rows x 2 columns]>

In [4]:
df['Disease'].value_counts().count()

24

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200 entries, 0 to 1199
Data columns (total 2 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Disease Description  1200 non-null   object
 1   Disease              1200 non-null   object
dtypes: object(2)
memory usage: 18.9+ KB


In [6]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

C:\Users\hp\anaconda3\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [7]:
token = tokenizer.encode_plus(
    df['Disease Description'].iloc[0],
    max_length=256,
    truncation=True,
    padding='max_length',
    add_special_tokens=True,
    return_tensors='tf'
)

In [8]:
token.input_ids

<tf.Tensor: shape=(1, 256), dtype=int32, numpy=
array([[  101, 21416,   117,  1573,  1874,  2922,   117,  3291,  6289,
          117,  3763, 12804,   117, 19569,  1116, 10536,  2489,   117,
         9322,  4943,   117, 16752, 20327, 11083, 10721,   119,   102,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0, 

In [9]:
x_input_ids = np.zeros((len(df), 256))
x_attn_masks = np.zeros((len(df), 256))

In [10]:
def generate_training_data(df, ids, masks, tokenizer):
    for i, text in tqdm(enumerate(df['Disease Description'])):
        tokenized_text = tokenizer.encode_plus(
            text,
            max_length=256,
            truncation=True,
            padding='max_length',
            add_special_tokens=True,
            return_tensors='tf'
        )
        ids[i, :] = tokenized_text.input_ids
        masks[i, :] = tokenized_text.attention_mask
    return ids, masks

In [11]:
X_input_ids, X_attn_masks = generate_training_data(df, x_input_ids, x_attn_masks, tokenizer)

0it [00:00, ?it/s]

In [12]:
labels = np.zeros((len(df), 24))
labels.shape

(1200, 24)

In [13]:
df.isnull().sum()

Disease Description    0
Disease                0
dtype: int64

In [14]:
# Define the class mappings
class_mapping = {
    'Avian Influenza': 0,
    'Rift Valley fever': 1,
    'Bovine tuberculosis': 2,
    'Lumpy skin ': 3,
    'Peste des Petits Ruminants': 4,
    'Bovine viral diarrhea': 5,
    'Newcastle': 6,
    'Infectious Coryza': 7,
    'Thrush': 8,
    'Mastitis': 9,
    'Tuberculosis': 10,
    'Bloat': 11,
    'Equine encephalomyelitis': 12,
    'Hemorrhagic septicemia': 13,
    'Brucellosis': 14,
    'Sheep Pox': 15,
    'Pneumonia': 16,
    'Foot and mouth disease': 17,
    'Bluetongue': 18,
    'Anaplasmosis': 19,
    'Rabies': 20,
    'Necrotic enteritis': 21,
    'Ascarids': 22,
    'Avian Pox': 23
}

# Apply the mapping
df['Disease'] = df['Disease'].map(class_mapping)

# Identify rows with unmapped (NaN) values
unmapped = df[df['Disease'].isna()]

# Display unmatched values
print("Unmapped disease names:")
print(unmapped['Disease'].unique())  # Shows the unique disease names that didn't map

# Optionally, handle these NaN values
# For example, fill NaNs with a default value or -1
df['Disease'] = df['Disease'].fillna(-1)

# If all values are mapped correctly and no NaNs are expected, convert to int
df['Disease'] = df['Disease'].astype(int)


Unmapped disease names:
[nan]


In [15]:
df.head()

,Disease Description,Disease
0,"Fever, Sore throat, Cough, Headache, Muscle p...",0
1,"fever , huffy throat , cough , headache , braw...",0
2,"fever , sensitive throat , cough , concern , m...",0
3,"fever , afflictive throat , cough , headache ,...",0
4,"fever , sore throat , cough , concern , muscle...",0


In [16]:
df.tail()

,Disease Description,Disease
1195,"Swollen eyelids eyelids , Eyes closed , Oral l...",23
1196,"Swollen eyelids , Eyes lesions closed , Oral l...",23
1197,"Swollen eyelids, Eyes closed, Oral lesions, We...",23
1198,"Swollen eyelids , Eyes Eyes closed , Oral lesi...",23
1199,"Swollen , Eyes closed , lesions , Weight loss ...",23


In [17]:
df.empty

False

In [18]:
# Apply class mapping to convert disease names to codes
df['Disease'] = df['Disease Description'].map(class_mapping)

# Handle any descriptions not in the mapping (assign them -1)
df['Disease'] = df['Disease'].fillna(-1)

# Convert the 'Disease' column to integers
df['Disease'] = df['Disease'].astype(int)

# Now create your one-hot encoded labels
labels[np.arange(len(df)), df['Disease'].values] = 1 # one-hot encoded target tensor

In [19]:
dataset = tf.data.Dataset.from_tensor_slices((x_input_ids, x_attn_masks, labels))
dataset.take(1) # one sample data

<_TakeDataset element_spec=(TensorSpec(shape=(256,), dtype=tf.float64, name=None), TensorSpec(shape=(256,), dtype=tf.float64, name=None), TensorSpec(shape=(24,), dtype=tf.float64, name=None))>

In [20]:

def AnimalDatasetMapFunction(input_ids, attn_masks, labels):
    return {
        'input_ids': input_ids,
        'attention_mask': attn_masks
    }, labels

In [21]:
dataset = dataset.map(AnimalDatasetMapFunction)

In [22]:
dataset.take(1)

<_TakeDataset element_spec=({'input_ids': TensorSpec(shape=(256,), dtype=tf.float64, name=None), 'attention_mask': TensorSpec(shape=(256,), dtype=tf.float64, name=None)}, TensorSpec(shape=(24,), dtype=tf.float64, name=None))>

In [23]:
dataset = dataset.shuffle(3500).batch(16, drop_remainder=True)

In [24]:
dataset.take(1)

<_TakeDataset element_spec=({'input_ids': TensorSpec(shape=(16, 256), dtype=tf.float64, name=None), 'attention_mask': TensorSpec(shape=(16, 256), dtype=tf.float64, name=None)}, TensorSpec(shape=(16, 24), dtype=tf.float64, name=None))>

In [25]:
p = 0.8
train_size = int((len(df)//16)*p)

In [26]:
train_size

60

In [27]:
train_dataset = dataset.take(train_size)
val_dataset = dataset.skip(train_size)

In [28]:
from transformers import TFBertModel

In [29]:
model = TFBertModel.from_pretrained('bert-base-cased')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [30]:
# defining 2 input layers for input_ids and attn_masks
input_ids = tf.keras.layers.Input(shape=(256,), name='input_ids', dtype='int32')
attn_masks = tf.keras.layers.Input(shape=(256,), name='attention_mask', dtype='int32')

bert_embds = model.bert(input_ids, attention_mask=attn_masks)[1] # 0 -> activation layer (3D), 1 -> pooled output layer (2D)
intermediate_layer = tf.keras.layers.Dense(512, activation='relu', name='intermediate_layer')(bert_embds)
output_layer = tf.keras.layers.Dense(24, activation='softmax', name='output_layer')(intermediate_layer) # softmax -> calcs probs of classes

animal_model = tf.keras.Model(inputs=[input_ids, attn_masks], outputs=output_layer)
animal_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_ids (InputLayer)      [(None, 256)]                0         []                            
                                                                                                  
 attention_mask (InputLayer  [(None, 256)]                0         []                            
 )                                                                                                
                                                                                                  
 bert (TFBertMainLayer)      TFBaseModelOutputWithPooli   1083102   ['input_ids[0][0]',           
                             ngAndCrossAttentions(last_   72         'attention_mask[0][0]']      
                             hidden_state=(None, 256, 7                                       

In [31]:
optim = tf.keras.optimizers.legacy.Adam(learning_rate=1e-5, decay=1e-6)
loss_func = tf.keras.losses.CategoricalCrossentropy()
acc = tf.keras.metrics.CategoricalAccuracy('accuracy')

In [32]:
animal_model.compile(optimizer=optim, loss=loss_func, metrics=[acc])

In [33]:
hist = animal_model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=2
)     

Epoch 1/5

60/60 [==============================] - 692s 11s/step - loss: 0.7319 - accuracy: 0.9062 - val_loss: 0.0255 - val_accuracy: 1.0000
Epoch 2/5
60/60 [==============================] - 668s 11s/step - loss: 0.0166 - accuracy: 1.0000 - val_loss: 0.0065 - val_accuracy: 1.0000
Epoch 3/5
 2/60 [>.............................] - ETA: 9:59 - loss: 0.0085 - accuracy: 1.0000 

KeyboardInterrupt: 

In [41]:
animal_model.save('animal_model.keras')

In [35]:
from tensorflow.keras.models import load_model

In [43]:
from tensorflow.keras.models import load_model
from transformers import TFBertMainLayer

# Define custom objects if any
custom_objects = {
    'TFBertMainLayer': TFBertMainLayer
}

# Load the model with custom objects
model = load_model('animal_model.keras', custom_objects=custom_objects)


ValueError: Expected object to be an instance of `KerasSaveable`, but got <tf_keras.src.engine.functional.Functional object at 0x0000021BC9111110> of type <class 'tf_keras.src.engine.functional.Functional'>